In [68]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

In [55]:
#Load training data
with open('../data/random-acts-of-pizza/train.json') as f:
    train_json = json.load(f)
    
# Load test data
with open('../data/random-acts-of-pizza/test.json') as f:
    test_json = json.load(f)

train = json_normalize(train_json)
test = json_normalize(test_json)

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)


Train Shape: (4040, 32)
Test Shape: (1631, 17)


In [66]:
train.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_received_pizza,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,0,1,False,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,False,[],0,1,0,1,None,nickylvst,1.317853e+09,1.317849e+09
1,N/A,2,5,False,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,False,"[AskReddit, Eve, IAmA, MontereyBay, RandomKind...",34,4258,116,11168,None,fohacidal,1.332652e+09,1.332649e+09
2,N/A,0,3,False,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,False,[],0,3,0,3,None,jacquibatman7,1.319650e+09,1.319646e+09
3,N/A,0,1,True,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,False,"[AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]",54,59,76,81,None,4on_the_floor,1.322855e+09,1.322855e+09
4,N/A,6,6,False,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,False,"[GayBrosWeightLoss, RandomActsOfCookies, Rando...",1121,1225,1733,1887,None,Futuredogwalker,1.373658e+09,1.373654e+09


In [67]:
train_data = train[:2800]
train_labels = train[:2800]['requester_received_pizza']
test_data = test[:]

dev_data = train[2800:]
dev_labels = train[2800:]['requester_received_pizza']

print('training label shape:', train_labels.shape)
print('dev label shape:',      dev_labels.shape)
print('training data shape:', train_data.shape)
print('test data shape:',      test_data.shape)
print('dev data shape:',     dev_data.shape)

training label shape: (2800,)
dev label shape: (1240,)
training data shape: (2800, 32)
test data shape: (1631, 17)
dev data shape: (1240, 32)


### Examine Data
---

 1. For first 5 training examples, print the title of request

In [54]:
def display_request(num_examples=5):

    for i in range(num_examples):
        print(train_data.iloc[i]['request_title']) 
        print(train_data.iloc[i]['request_text'])
        print('Received Pizza: ', train_data.iloc[i]['requester_received_pizza']) 
        print('\n')  

        
display_request(5)

Request Colorado Springs Help Us Please
Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated
Received Pizza:  False


[Request] California, No cash and I could use some dinner
I spent the last money I had on gas today. Im broke until next Thursday :(
Received Pizza:  False


[Request] Hungry couple in Dundee, Scotland would love some pizza!
My girlfriend decided it would be a good idea to get off at Perth bus station when she was coming to visit me and has since had to spend all her money on a taxi to get to me here in Dundee. Any chance some kind soul would get us some pizza since we don't have any cash anymore?
Received Pizza:  False


[Request] In Canada (Ontario), just got home from school. Need pizza.
It's cold, I'n hungr

In [85]:
print('*** 1. Unigram Feature Vectors ***')
title_train_data = train_data['request_title']
vectorizer = CountVectorizer()
vector = vectorizer.fit_transform(title_train_data)

#What is the size of the vocabulary?
print('Size of vocabulary: ', vector.shape[1])
print("0th feature: ", vectorizer.get_feature_names()[0])
print("Last feature: ", vectorizer.get_feature_names()[-1])
print("\n")


vectorizer2 = CountVectorizer(ngram_range=(2, 2))
vector2 = vectorizer2.fit_transform(title_train_data)
print('Size of vocabulary: ', vector2.shape[1])
print("0th feature: ", vectorizer2.get_feature_names()[0])
print("Last feature: ", vectorizer2.get_feature_names()[-1])


print('\n*** 4. Prune words that appear in fewer than 10 documents ***')
vectorizer = CountVectorizer(min_df=10)
vector = vectorizer.fit_transform(title_train_data)
print('Size of vocabulary: ', vector.shape[1])
print("0th feature: ", vectorizer.get_feature_names()[0])
print("Last feature: ", vectorizer.get_feature_names()[-1])
vocab = vectorizer.vocabulary_ 
print(vocab)
print("\n")

*** 1. Unigram Feature Vectors ***
Size of vocabulary:  3672
0th feature:  000
Last feature:  ಠ_ಠ


Size of vocabulary:  16072
0th feature:  000 bucks
Last feature:  zucchini and

*** 4. Prune words that appear in fewer than 10 documents ***
Size of vocabulary:  441
0th feature:  able
Last feature:  your
{'request': 312, 'colorado': 77, 'help': 162, 'us': 399, 'please': 296, 'california': 58, 'no': 256, 'cash': 64, 'and': 16, 'could': 82, 'use': 401, 'some': 336, 'dinner': 94, 'hungry': 177, 'couple': 84, 'in': 183, 'would': 434, 'love': 218, 'pizza': 293, 'canada': 60, 'ontario': 273, 'just': 193, 'got': 149, 'home': 168, 'from': 134, 'school': 322, 'need': 248, 'old': 269, 'friend': 132, 'coming': 79, 'to': 382, 'feed': 113, 'her': 163, 'indiana': 184, 'll': 211, 'give': 144, 'two': 392, 'week': 417, 'live': 209, 'for': 125, 'slice': 333, 'of': 264, 'pie': 292, 'me': 227, 'back': 36, 'my': 244, 'on': 270, 'friday': 130, 'random': 303, 'have': 158, 'if': 179, 'nothing': 260, 'the': 36

In [98]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

def text_preprocessor(text):
    text = text.lower()
    text = re.sub("\W",' ',text) # replace non-alphanumeric
    text = re.sub("_",' ',text) # replace non-alphanumeric
    text = re.sub('\n', '', text)   
    return text

print('\n***  Naive Bayes models ***') 
title_train_data = train_data['request_title']
title_dev_data = dev_data['request_title']
vectorizer = CountVectorizer(stop_words='english', preprocessor=text_preprocessor)
train_vector = vectorizer.fit_transform(title_train_data)

dev_vector = vectorizer.transform(title_dev_data)

mnb = MultinomialNB(alpha=0.01)
mnb.fit(train_vector, train_labels)

#Evaluate performance on the dev set.
pred_mnb = mnb.predict(dev_vector)
score = metrics.f1_score(dev_labels, pred_mnb, average="weighted")
print(f"f1 score = {score}")
for i, row in enumerate(dev_vector):
    if(pred_mnb[i]==True):
        print(i, pred_mnb[i])
        
        



***  Naive Bayes models ***
f1 score = 0.6721260018261931
10 True
13 True
21 True
23 True
31 True
33 True
36 True
39 True
41 True
45 True
80 True
83 True
84 True
85 True
98 True
106 True
116 True
120 True
141 True
144 True
155 True
157 True
164 True
167 True
176 True
182 True
186 True
202 True
210 True
212 True
224 True
250 True
258 True
262 True
265 True
267 True
268 True
273 True
277 True
279 True
282 True
287 True
288 True
303 True
304 True
308 True
309 True
311 True
312 True
330 True
340 True
341 True
353 True
355 True
374 True
378 True
383 True
386 True
389 True
395 True
399 True
412 True
418 True
420 True
429 True
430 True
431 True
432 True
436 True
438 True
448 True
455 True
463 True
464 True
468 True
480 True
483 True
485 True
486 True
489 True
490 True
491 True
494 True
505 True
513 True
521 True
522 True
526 True
539 True
545 True
546 True
555 True
568 True
570 True
573 True
574 True
577 True
578 True
594 True
606 True
611 True
617 True
625 True
632 True
633 True
634 True
64

In [100]:
dev_data.iloc[10]

giver_username_if_known                                                                               N/A
number_of_downvotes_of_request_at_retrieval                                                             1
number_of_upvotes_of_request_at_retrieval                                                               0
post_was_edited                                                                                     False
request_id                                                                                      t3_18zjcg
request_number_of_comments_at_retrieval                                                                 0
request_text                                            Was planning to go grocery shopping today but ...
request_text_edit_aware                                 Was planning to go grocery shopping today but ...
request_title                                           [Request] Check has not cleared yet. North hol...
requester_account_age_in_days_at_request      

In [82]:
# Understand nature of the data .info() .describe()
# Histograms and boxplots 
# Value counts 
# Missing data 
# Correlation between the metrics 
# Explore interesting themes 
    # Wealthy survive? 
    # By location 
    # Age scatterplot with ticket price 
    # Young and wealthy Variable? 
    # Total spent? 
# Feature engineering 
# preprocess data together or use a transformer? 
    # use label for train and test   
# Scaling?

# Model Baseline 
# Model comparison with CV 

In [9]:
train.describe()

,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,request_number_of_comments_at_retrieval,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,...,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,unix_timestamp_of_request,unix_timestamp_of_request_utc
count,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,...,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4040.000000,4.040000e+03,4.040000e+03,4.040000e+03,4.040000e+03
mean,2.424505,6.180446,2.871040,254.586579,757.692720,16.417034,518.993205,115.098267,289.425743,0.645050,...,41.151733,0.063614,1.239109,18.076733,1160.079950,2720.342079,3.743236e+03,7.788069e+03,1.342829e+09,1.342826e+09
std,3.023101,10.746320,4.723339,303.275730,333.035728,70.651428,267.872623,193.318968,357.416133,3.413813,...,80.798543,0.325773,0.603083,21.736465,3718.365515,6264.378878,2.583816e+04,3.916741e+04,2.333057e+07,2.332989e+07
min,0.000000,0.000000,0.000000,0.000000,45.291562,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-173.000000,-173.000000,0.000000e+00,0.000000e+00,1.297723e+09,1.297723e+09
25%,1.000000,2.000000,0.000000,3.473168,522.248455,0.000000,279.009051,0.000000,8.000000,0.000000,...,2.000000,0.000000,1.000000,1.000000,3.000000,22.000000,9.000000e+00,5.200000e+01,1.320469e+09,1.320466e+09
50%,2.000000,4.000000,1.000000,157.067170,753.270874,0.000000,528.781939,24.000000,114.000000,0.000000,...,13.000000,0.000000,1.000000,11.000000,174.500000,708.000000,3.510000e+02,1.283500e+03,1.342565e+09,1.342561e+09
75%,3.000000,7.000000,4.000000,390.092653,900.349838,0.000000,776.226670,140.250000,479.000000,0.000000,...,46.000000,0.000000,1.000000,27.000000,1163.750000,3304.000000,2.303750e+03,6.829000e+03,1.364618e+09,1.364614e+09
max,47.000000,345.000000,61.000000,2809.750787,2879.276319,785.457685,1025.407593,994.000000,1000.000000,88.000000,...,999.000000,5.000000,9.000000,186.000000,155010.000000,223708.000000,1.286864e+06,2.046482e+06,1.381552e+09,1.381523e+09


In [3]:
train.head()

,giver_username_if_known,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_id,request_number_of_comments_at_retrieval,request_text,request_text_edit_aware,request_title,requester_account_age_in_days_at_request,...,requester_received_pizza,requester_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,requester_user_flair,requester_username,unix_timestamp_of_request,unix_timestamp_of_request_utc
0,N/A,0,1,False,t3_l25d7,0,Hi I am in need of food for my 4 children we a...,Hi I am in need of food for my 4 children we a...,Request Colorado Springs Help Us Please,0.000000,...,False,[],0,1,0,1,None,nickylvst,1.317853e+09,1.317849e+09
1,N/A,2,5,False,t3_rcb83,0,I spent the last money I had on gas today. Im ...,I spent the last money I had on gas today. Im ...,"[Request] California, No cash and I could use ...",501.111100,...,False,"[AskReddit, Eve, IAmA, MontereyBay, RandomKind...",34,4258,116,11168,None,fohacidal,1.332652e+09,1.332649e+09
2,N/A,0,3,False,t3_lpu5j,0,My girlfriend decided it would be a good idea ...,My girlfriend decided it would be a good idea ...,"[Request] Hungry couple in Dundee, Scotland wo...",0.000000,...,False,[],0,3,0,3,None,jacquibatman7,1.319650e+09,1.319646e+09
3,N/A,0,1,True,t3_mxvj3,4,"It's cold, I'n hungry, and to be completely ho...","It's cold, I'n hungry, and to be completely ho...","[Request] In Canada (Ontario), just got home f...",6.518438,...,False,"[AskReddit, DJs, IAmA, Random_Acts_Of_Pizza]",54,59,76,81,None,4on_the_floor,1.322855e+09,1.322855e+09
4,N/A,6,6,False,t3_1i6486,5,hey guys:\n I love this sub. I think it's grea...,hey guys:\n I love this sub. I think it's grea...,[Request] Old friend coming to visit. Would LO...,162.063252,...,False,"[GayBrosWeightLoss, RandomActsOfCookies, Rando...",1121,1225,1733,1887,None,Futuredogwalker,1.373658e+09,1.373654e+09


In [5]:
train.shape

(4040, 32)

In [11]:
pizza = train[train['requester_received_pizza']==True]
no_pizza = train[train['requester_received_pizza']==False]
print(len(pizza))
print(len(no_pizza))

994
3046


In [12]:
pizza.describe()

,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,request_number_of_comments_at_retrieval,requester_account_age_in_days_at_request,requester_account_age_in_days_at_retrieval,requester_days_since_first_post_on_raop_at_request,requester_days_since_first_post_on_raop_at_retrieval,requester_number_of_comments_at_request,requester_number_of_comments_at_retrieval,requester_number_of_comments_in_raop_at_request,...,requester_number_of_posts_at_retrieval,requester_number_of_posts_on_raop_at_request,requester_number_of_posts_on_raop_at_retrieval,requester_number_of_subreddits_at_request,requester_upvotes_minus_downvotes_at_request,requester_upvotes_minus_downvotes_at_retrieval,requester_upvotes_plus_downvotes_at_request,requester_upvotes_plus_downvotes_at_retrieval,unix_timestamp_of_request,unix_timestamp_of_request_utc
count,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,...,994.000000,994.000000,994.000000,994.000000,994.000000,994.000000,9.940000e+02,9.940000e+02,9.940000e+02,9.940000e+02
mean,2.566398,8.101610,5.261569,276.766844,828.179075,28.563757,579.609191,126.528169,355.248491,1.429577,...,47.989940,0.141851,1.725352,19.509054,1372.060362,3572.609658,4.923700e+03,1.094121e+04,1.338609e+09,1.338606e+09
std,3.390663,13.085617,5.800128,312.791210,353.852857,96.767955,262.029770,202.328504,377.352464,5.914937,...,89.005244,0.526249,0.900946,21.352374,5419.691555,9161.651778,4.216637e+04,6.819221e+04,2.317988e+07,2.317956e+07
min,0.000000,0.000000,0.000000,0.000000,45.291562,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-39.000000,-9.000000,0.000000e+00,2.000000e+00,1.306095e+09,1.306091e+09
25%,1.000000,3.000000,2.000000,39.110437,604.934381,0.000000,352.477613,1.000000,31.250000,0.000000,...,4.000000,0.000000,1.000000,3.000000,23.250000,139.000000,4.500000e+01,2.722500e+02,1.316329e+09,1.316325e+09
50%,2.000000,5.000000,4.000000,179.528218,821.140781,0.000000,622.100469,35.000000,187.500000,0.000000,...,17.000000,0.000000,2.000000,13.000000,285.500000,1207.000000,5.620000e+02,2.272000e+03,1.336384e+09,1.336381e+09
75%,3.000000,9.000000,7.000000,396.927688,978.019358,0.000000,827.571742,155.000000,688.500000,0.000000,...,53.000000,0.000000,2.000000,28.000000,1382.750000,4344.500000,2.937250e+03,8.871250e+03,1.359247e+09,1.359247e+09
max,35.000000,276.000000,61.000000,2124.362650,2733.297060,785.457685,928.507593,990.000000,1000.000000,88.000000,...,987.000000,5.000000,9.000000,128.000000,155010.000000,223708.000000,1.286864e+06,2.046482e+06,1.381297e+09,1.381268e+09


In [33]:
print(no_pizza["requester_account_age_in_days_at_request"].mean())
print(pizza["requester_account_age_in_days_at_request"].mean())
print('\nrequest_number_of_comments_at_retrieval')
print(no_pizza["request_number_of_comments_at_retrieval"].mean())
print(pizza["request_number_of_comments_at_retrieval"].mean())
print('\nnumber_of_upvotes_of_request_at_retrieval')
print(no_pizza["number_of_upvotes_of_request_at_retrieval"].mean())
print(pizza["number_of_upvotes_of_request_at_retrieval"].mean())
print('\nrequester_number_of_comments_in_raop_at_request')
print(no_pizza["requester_number_of_comments_in_raop_at_request"].mean())
print(pizza["requester_number_of_comments_in_raop_at_request"].mean())
print('\nrequester_number_of_posts_at_retrieval')
print(no_pizza["requester_number_of_posts_at_retrieval"].mean())
print(pizza["requester_number_of_posts_at_retrieval"].mean())
print('\nrequester_number_of_subreddits_at_request')
print(no_pizza["requester_number_of_subreddits_at_request"].mean())
print(pizza["requester_number_of_subreddits_at_request"].mean())

no_pizza["requester_user_flair"]

247.34850245692647
276.7668435031487

request_number_of_comments_at_retrieval
2.0909389363099145
5.261569416498994

number_of_upvotes_of_request_at_retrieval
5.553512803676953
8.101609657947686

requester_number_of_comments_in_raop_at_request
0.3890347997373605
1.4295774647887325

requester_number_of_posts_at_retrieval
38.92022324359816
47.989939637826964

requester_number_of_subreddits_at_request
17.609323703217335
19.509054325955734


0       None
1       None
2       None
3       None
4       None
        ... 
4034    None
4035    None
4037    None
4038    None
4039    None
Name: requester_user_flair, Length: 3046, dtype: object

In [28]:
print(len(pizza["request_title"])

5       [REQUEST] I'll give a two week xbox live code ...
9               [REQUEST]We're in need of some om noms...
10      [REQUEST] Bummed out in Chicago. Too broke to ...
16      [request] Cookeville, TN. My dog recently died...
18      [Request] Virginia. Girlfriend and I our sick,...
                              ...                        
4004    [request] broken shoulder, out of work, very h...
4008    [Request] Atlanta, Georgia Suffered from a sad...
4012    [REQUEST] Essex Junction, VT - Nearly broke, n...
4024    [request] Unemployment check never came, waiti...
4036    [Request][USA] Papa Johns is giving away one f...
Name: request_title, Length: 994, dtype: object
